# Naiwny klasyfikator Bayesa

[Naiwny klasyfikator Bayesa](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) został zaprezentowany na ostatnim wykładzie. Słowo ,,naiwny'' oznacza, że wszystkie cechy wejściowe są od siebie niezależne. W praktyce to założenie nie jest nigdy spełnione, ale cały czas klasyfikator może działać bardzo dobrze.

Dzisiejsze zajecia będą polegać na implementacji klasyfikatora, którą później wykorzystamy do zbudowania filtra antyspamowego. Filtr antyspamowy to nic innego niż klasyfikator, który próbuje przewidzieć dla danej wiadomości jedną z dwóch klas: *SPAM* lub *NO_SPAM*.

**zadanie 0** Ściągnij zbiór danych z [link](https://www.dropbox.com/s/g692nz2yx7dlexe/mailbox.tgz?dl=0) i rozpakuj go w katalogu z tym notebookiem. 

Będziemy reprezentować klasyfikator jako następującą klasę ``NaiveBayes``:
 * `class_count`: słownik zawierający ile razy dana klasa wystąpiła w zbiorze treningowym
 * `feature_count`: słownik zawierający informacje ile razy dana cecha współwystąpiła z podaną klasą.
 * `get_features`: funkcja, która zwróci zbiór cech.

In [ ]:
class NaiveBayes:

    """
    Klasa reprezentująca parametry i stan naiwnego klasyfikatora
    bayesowskiego.
    """

    def __init__(self, get_features):
        self.feature_count = { }
        self.class_count = { }
        self.get_features = get_features


**zadanie 1** 
Napisz funkcję `train(bayes, item, cat)`, która zlicza:
 * ile razy pojawiła się dana kategoria `cat` oraz
 * ile razy cechy dokumentu `item` współwystępowały z daną categorią.
 
Funkcja ma korzystać z metody `bayes.get_features(item)`, żeby uzyskać zbiór cech. Należy zwiększać słownik `bayes.feature_count` dla każdej pary cechy i kategorii o jeden. W słowniku `bayes.class_count` należy zliczać, ile dokumentów należy do danej kategorii.

**zadanie 2**
Napisz funkcję `featprob(bayes, feature, category)`, która oblicza prawdopodobieństwo $P(F=f|C=c)$ dla danej cechy (`feature`) i danej kategorii (`category`). Prawodpodobieństwo warunkowe obliczamy, dzieląc liczbę współwystępowania cechy `feature` z kategorią `category` przez liczbę występowania kategorii `category`. Funkcja zwraca prawdopodobieństwo w postaci logarytmu. Jeżeli dana kategoria nie istnieje, zwracamy -1e300 (liczba możliwie bliska $log(0) = -inf$). Jeśli nie istnieje dana cecha (dla danej kategorii), to zwracamy logarytm ilorazu liczby $0.001$ oraz liczby występowania kategorii (wygładzanie). 

**zadania 3** Napisz funkcję `catprob(bayes, category)`, która oblicza logarytm
prawodpodobieństwa $P(C=c)$. Prawdopodobieństwo to obliczamy dzieląc liczbę wystąpień danej kategorii category przez sumę występowań wszystkich kategorii. Jeżeli kategoria nie istnieje należy zwrócić liczbę $-1e300$ (liczba możliwie bliska $log(0) = -inf$)

**zadanie 4** Napisz funkcję `docprob(bayes, item, cat)`, która oblicza uogólnioną sumę z równania:
$$
P(item, cat) = P(cat) \cdot \prod_{i = 0}^{n}{P(feature_i|cat)}
$$
gdzie $feature_i$ jest $i$-tą cechą w dokumencie ``item``.

**zadanie 5** Napisz funkcję `classify(bayes, item)`, która wykorzystuje funkcję `docprob` do obliczenia największej wartosci $P(category|document)$ dla dokumentu *item*. Należy zwrócić najbardziej prawdopodobną kategorię.

## Trenowanie filtra antyspamowego

Mając zaimplementowany klasyfikator Bayesowski, możemy uruchomić go zbiorze danych z zadania 0. Poniższe dwie funkcje mają charakter pomocniczy:
 * `getwords`: zwraca listę cech z danego dokumentu
 * `category`: zwraca klasę (*spam*, *no-spam*) z nazwy dokumentu

In [ ]:
import glob
import re

def getwords(docname):
    """Wyznacza zbiór cech (słów)."""
    doc = open(docname).read()
    splitter = re.compile('\\W*')
    words = [s for s in splitter.split(doc)]
    return set(words)

def category(name):
    """Zwraca etykietę kategorii."""
    if "spm" in name:
        return "SPAM"
    else:
        return "NO-SPAM"

Funkcja `train_model` wczytuje dane i zwraca wytrenowany klasyfikator.

In [ ]:
def train_model(directory, verbose=False):
    """Dokonuje sprawdzenia"""

    trainlist = []
    for i in range(1, 9):
        trainlist.extend(glob.glob("%s/part%d/*" % (directory, i)))

    classifier = NaiveBayes.NaiveBayes(getwords)

    if verbose:
        print(i, "\tTraining classifier")
    for doc in trainlist:
        train(classifier, doc, category(doc))

    return classifier

In [ ]:
def valid(classifier, directory, part):
    correct = 0
    total = 0

    validlist = []
    validlist.extend(glob.glob("%s/part%d/*" % (directory, part)))
    for doc in validlist:
        bestcat = classify(classifier, doc)
        if bestcat == category(doc):
            correct += 1
        total += 1
    return float(correct) / float(total)

Trenujemy model:

In [ ]:
model = train_model("mailbox", True)

I testujemy go na zbiorze walidacyjnym i testowym:

In [ ]:
print("Accuracy-DEV", valid(model, "mailbox", 9))
print("Accuracy-TEST", valid(model, "mailbox", 10))